In [1]:
# https://spacy.io/usage/vectors-similarity

# install spacy
# install torchtext

import spacy
import torchtext
import json

# python -m spacy download
nlp = spacy.load('en_core_web_lg')
spacy.load('en_vectors_web_lg', vocab=nlp.vocab)

# Have project be a few scripts to run to avoid having everything in one environment
# (also that would be a lot in memory)
# (but mainly I don't want to put everything on one environment rn)

C:\Users\Joseph\Anaconda3\envs\nlp\lib\runpy.py:193: UserWarning: [W019] Changing vectors name from en_core_web_lg.vectors to en_core_web_lg.vectors_1070971, to avoid clash with previously loaded vectors. See Issue #3853.
  "__main__", mod_spec)


In [2]:
# get lists
with open('results.json', 'r') as it:
    results = json.load(it)

with open('failed.json', 'r') as it:
    failed = json.load(it)

# Some Testing Stuff

In [3]:
tokens = nlp('bicycle human top on')

for token in tokens:
    print(token.text, token.has_vector, token.vector_norm, token.is_oov)

bicycle True 6.655211 True
human True 6.869779 True
top True 5.5138364 True
on True 5.218119 True


In [4]:
[print(token1.text, token2.text, token1.similarity(token2)) for token1 in tokens for token2 in tokens if token1 != token2];

bicycle human 0.18300328
bicycle top 0.16640997
bicycle on 0.233741
human bicycle 0.18300328
human top 0.18022928
human on 0.25648427
top bicycle 0.16640997
top human 0.18022928
top on 0.385453
on bicycle 0.233741
on human 0.25648427
on top 0.385453


In [5]:
text = "Human is on bike. Human is on top of bike."
hi = nlp(text)
sentences = list(hi.sents) # list of sentences

for sent in sentences:
    print(sent)
sent.sentiment

Human is on bike.
Human is on top of bike.


0.0

In [6]:
words = set()
for row in results: # grab each word from results
    words.add(row[0])
    words.add(row[1])
words

{'bottle', 'person', 'pizza'}

In [7]:
nlp(results[0][0]).similarity(nlp(results[0][1]))

0.2141891835424058

# Trying to find the 'best' sentence

In [8]:
# Construct the "best" sentence
for i in range(len(results)):

    word1 = nlp(results[i][0])
    word2 = nlp(results[i][1])

    prep_similarity = {}
    for prep in results[i][4]:
        #print(prep)
        nlprep = nlp(prep)

        avg = nlprep.similarity(word1) + nlprep.similarity(word2)
        avg = avg / 2

        #print("{:.4f}".format(avg))
        prep_similarity[prep] = avg

    prep = max(prep_similarity, key = prep_similarity.get)

    # might just want the first one
    if word1.similarity(nlp(prep)) > word2.similarity(nlp(prep)):
        sentence = word1.text + " " + prep + " " + word2.text
    else:
        sentence = word2.text + " " + prep + " " + word1.text
    print(sentence)

person is to the right of pizza
person is to the left of pizza
bottle is beside pizza
person is to the left of person
person is to the right of person
person is holding bottle
person is holding bottle


In [9]:
# Or pick the "best" sentence ... not sure how yet
# just getting all sentences right now
i = 0

sentences = []
for prep in results[i][4]:
    sentences.append(word1.text + " " + prep + " " + word2.text)
    sentences.append(word2.text + " " + prep + " " + word1.text)

nl = nlp(". ".join(sentences))
sents = list(nl.sents)
sents

[bottle is beside person.,
 person is beside bottle.,
 bottle is to the,
 right of person.,
 person is to the,
 right of bottle.,
 bottle is adjacent to person.,
 person is adjacent to bottle.,
 bottle is holding person.,
 person is holding bottle.,
 bottle overlaps person.,
 person overlaps bottle]